In [159]:
import sys
import pandas as pd
import altair as alt
import numpy as np
import pprint
import datetime as dt
from vega_datasets import data

In [160]:
df = pd.read_csv("https://raw.githubusercontent.com/Pullze/kpop-info-vis/refs/heads/master/data/billboard_200_cleaned.csv")

In [161]:
df.head()


,date,Artist,Song/Album Title,Peak Position,Weeks on Chart,year,generation,debut_years
0,2009-04-04,BoA,BoA[6],127,1,2009,1,1996 - 2004
1,2012-03-17,Big Bang,Alive[7][8],150,1,2012,2,2005 - 2011
2,2012-05-12,Girls' Generation-TTS,Twinkle[9][10],126,1,2012,2,2005 - 2011
3,2012-09-29,G-Dragon,One of a Kind[8],161,1,2012,2,2005 - 2011
4,2013-09-21,G-Dragon,Coup d'Etat Pt.1[8],182,1,2013,2,2005 - 2011


In [162]:
# Remove citations like [8] from the column
df['Song/Album Title'] = df['Song/Album Title'].str.replace(r'\[\d+\]', '', regex=True)

In [163]:
df.head()

,date,Artist,Song/Album Title,Peak Position,Weeks on Chart,year,generation,debut_years
0,2009-04-04,BoA,BoA,127,1,2009,1,1996 - 2004
1,2012-03-17,Big Bang,Alive,150,1,2012,2,2005 - 2011
2,2012-05-12,Girls' Generation-TTS,Twinkle,126,1,2012,2,2005 - 2011
3,2012-09-29,G-Dragon,One of a Kind,161,1,2012,2,2005 - 2011
4,2013-09-21,G-Dragon,Coup d'Etat Pt.1,182,1,2013,2,2005 - 2011


In [164]:
# The dataframe is the frequence of the genreation performance in the billboard 200
gen_count = df.pivot_table(index='generation', columns='year', values='Weeks on Chart', aggfunc='count', fill_value=0)

## Graph_4th_gen

### Timeline of Billboard 200 - highlight 2009, 2018 and 2023

### Annotations to Insert:



1. In 2023, 35 albums reached the Billboard 200, with five climbing to the number one spot. NewJeans' Get Up held the top position the longest, staying on the chart for 26 weeks. Other notable top albums include:

- The Name Chapter: Temptation - TXT
- 5-Star - Stray Kids
- Rock-Star - Stray Kids
- The World EP.Fin: Will - Ateez

<!-- 2. Except for Billboard, Spotify’s Billions Club, a prestige playlist with tracks that have over a billion streams, continues to welcome K-pop songs. In 2023, BTS member Jimin’s “Like Crazy and BTS member Jungkook’s “Seven” joined the club. Jungkook’s “Seven” reached the one-billion-streams threshold after just 108 days since its release, making it the fastest track in Spotify’s history to join the club. -->


❗️❗️❗️ TODO: 


2. Insert: https://youtube.com/clip/UgkxE2AcnHpwSbmlsZqO9MjsoHsgFO8V_fuj?si=QxrdC95oGYHu1Lw0








In [165]:
columns = ['Song/Album Title', 'Artist', 'Peak Position', 'Weeks on Chart', 'year', 'generation']
df = df[columns]
df['year'] = pd.to_datetime(df['year'], format='%Y')

In [ ]:

#TODO: replicate vis3


# Create a selection and condition for the vertical line, annotation dots, and text annotations
hover = alt.selection_point(on='mousemove', nearest=True, fields=['year'], empty='none')

# Create the base chart and filter to All polls
base = alt.Chart(df).transform_aggregate(
    total_songs='count()',
    groupby=['year', 'generation']
).transform_filter(
    alt.datum.total_songs > 0
).mark_bar(size=25).encode(
    x=alt.X('year:T', title=None, axis=alt.Axis(
        format='%Y',
        labelAngle=-45,     # Rotate labels for readability
        labelOverlap='greedy'  # Automatically manage overlapping labels
    )),
    y=alt.Y(
        'total_songs:Q',
        title='The Number of Songs/Albums',
        scale=alt.Scale(nice=True)  # Automatically adjust the range of y-axis
    ),
    color='generation:N'
).properties(
    width=600,
    height=500
)

# Static line chart
line_chart = base

# Vertical line
vline = base.mark_rule(size=25, color='lightgray').encode(
    x='year:T',
    opacity=alt.condition(hover, alt.value(1), alt.value(0))
).add_params(
    hover
)

#interaction dots
interaction_dots = base.mark_circle(size=90).encode(
    x='year:T',
    y='total_songs:Q',
    color='generation:N',
    opacity=alt.condition(hover, alt.value(1), alt.value(0))
).transform_filter(
    hover
)

#interaction text labels
text_labels = base.mark_text(fontSize=14, align='left', dx=7).encode(
    x='year:T',
    y='total_songs:Q',
    text=alt.Text('total_songs:Q', format='.0f'),
    color='generation:N',
    opacity=alt.condition(hover, alt.value(1), alt.value(0))
).transform_filter(
    hover
)


# Create a bar chart to show the average weeks on chart by generation
bars = alt.Chart(df).mark_bar().encode(
        x=alt.X(
        'mean(Weeks on Chart):Q', 
        title='Average Weeks on Chart',
            axis=alt.Axis(
            format='.0f',  # Show integers only
            tickCount=5    # Limit the number of ticks
        ),
        scale=alt.Scale(domain=[0, 14])  # Set x-axis range to match your data
    ),
    y=alt.Y('generation:N', sort='x', title='Generation'),
    color=alt.Color(
        'generation:N',  # Align bar colors with generations
        scale=alt.Scale(scheme='category10'),  # Use a consistent color scheme
        legend=None  # Hide the legend if it's not needed
    ),
    tooltip=['generation:N', 'mean(Weeks on Chart):Q']
).transform_window(
    rank='rank(mean(Weeks on Chart))',
    sort=[alt.SortField('mean(Weeks on Chart)', order='descending')]
).transform_filter(
    hover
).properties(
    title='Average Weeks on Chart by Generation'
)

# Combine charts (if needed)

#Put them all together
chart = alt.layer(line_chart, vline, interaction_dots, text_labels).properties(
    title={
        "text": "The Number of K-pop Albums that Hit Billboard 200 by Year",
        "subtitle": "Grouped by Generation",
        "fontSize": 16,
        "font": "Arial",
        "anchor": "middle",
    }
)

chart

/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/vegalite/v5/api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [145]:
# Combine the choropleth map and bar chart
combined_chart = alt.vconcat(chart, bars)

combined_chart


/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)

In [146]:

# # Create a custom color palette
# colors = ['gray' if year < 2017 or year > 2023 else sns.color_palette('Set2')[i % 8] for i, year in enumerate(year_total.index)]


# plt.figure(figsize=(12, 6))
# sns.barplot(x=year_total.index, y=year_total['Weeks on Chart'], palette=colors)

# plt.title('The Number of K-pop Albums that Hit Billboard 200 by Year')
# plt.xlabel('Year')
# plt.ylabel('The Number of K-pop Albums')
# plt.xticks(rotation=45)

# # Remove ticks
# plt.tick_params(left=False, bottom=False)
# plt.grid(False)

# # Remove the border
# sns.despine()

# plt.show()

In [147]:
artist_total = df.pivot_table(index='Artist',  values='Weeks on Chart', aggfunc='count', fill_value=0)



In [148]:
artist_total = artist_total.reset_index()
artist_total = artist_total.rename(columns={'Weeks on Chart': 'num'})

In [149]:
df = artist_total
# Create generation based on artist name
df['generation'] = 0
df['generation'] = np.where(df['Artist'] == 'BoA', 1, df['generation'])
df['generation'] = np.where(df['Artist'].isin(['Big Bang', '2NE1', "Girls' Generation-TTS", "Girls' Generation", 'Taeyang', 'G-Dragon', "Jackson Wang", 'Jonghyun']), 2, df['generation'])
df['generation'] = np.where(df['Artist'].isin(['Exo', 'BTS', "Blackpink", "Exo-K", 'The Rose', 'Seventeen', 'Twice', 'Red Velvet', 'Jimin', 'Jungkook', 'NCT 127', 'NCT Dream', 'NCT', 'J-Hope', 'Loona', 'Nayeon', 'Agust D', 'Jihyo', 'DPR Ian', 'SuperM', 'Monsta X', 'RM', 'V']), 3, df['generation'])
df['generation'] = np.where(df['Artist'].isin(['Stray Kids', '(G)I-dle', 'TXT', 'Itzy', 'Ateez', 'Aespa', 'NewJeans', 'Enhypen', 'Le Sserafim', 'Nmixx', 'P1Harmony', 'Kep1er' ]), 4, df['generation'])
df['generation'] = np.where(df['Artist'].isin(['Babymonster', 'Illit', 'BoyNextDoor', 'Xikers']), 5, df['generation'])



In [150]:
df = df.sort_values(by='num', ascending=False)

In [151]:
df.head()

,Artist,num,generation
5,BTS,16,3
42,TXT,11,4
31,NCT 127,8,3
39,Seventeen,8,3
12,Enhypen,7,4


In [152]:
df.to_csv('../data/generation_rank.csv', )

## Graph_3&4_gen

### Annotations to insert

- Among the Top 20 K-pop groups and singers with the most Billboard 200 hits, nine are from the 3rd generation, with Twice and Blackpink being the only girl groups. Another nine belong to the 4th generation, evenly divided between three boy bands and three girl groups. The 5th-generation group BoyNextDoor and the 2nd-generation artist G-Dragon each have one entry on the list.

 


In [153]:
df.sample(10)

,Artist,num,generation
27,Le Sserafim,4,4
36,P1Harmony,3,4
40,Stray Kids,5,4
24,Jonghyun,1,2
35,Nmixx,2,4
21,Jackson Wang,1,2
12,Enhypen,7,4
25,Jungkook,1,3
32,NCT Dream,2,3
6,Babymonster,1,5


In [154]:
options = [1, 2, 3, 4, 5]
labels = [str(option) for option in options]

input_dropdown = alt.binding_select(options=options + [None], labels=labels + ['All'], name='Generation ')
selection = alt.selection_point(fields=['generation'], bind=input_dropdown, value=None)

chart = alt.Chart(
    df,
    title=alt.Title(text="Top 20 K-pop Groups with the Most Billboard 200 Hits (2009-2023)", fontSize=18)
).mark_bar().encode(
    x=alt.X('Artist:N', sort='-y', title=None, axis=alt.Axis(
        labelAngle=-45  # Rotate labels for readability
    )),
    y=alt.Y('num:Q', title='Frequency of Hitting Billboard 200'),
    color=alt.Color(
        'generation:N',
        title='Generation',
        scale=alt.Scale(
            domain=[1, 2, 3, 4, 5],  # Explicitly define the generations
            scheme='tableau20'
        )
    ),
    tooltip=[
        alt.Tooltip('num:Q', title='Frequency'),  # Show frequency
        alt.Tooltip('generation:N', title='Generation'),
    ]
).add_params(
    selection
).transform_filter(
    selection
).transform_window(
    rank='rank()',
    sort=[]
).transform_filter(
    (alt.datum.rank <= 20)
).properties(
    width=600,
    height=400
).configure_view(
    strokeWidth=0
)

chart

/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [155]:
chart.save('gen_rank.json')

/home/lai/anaconda3/envs/vis/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
